[IRIS]

- Feature : 4개 Sepal_Length, Sepal_Width, Petal_Width, Petal_Length
- Target : 1개
- Class : 품종 2개만 선택 <- Setosa와 나머지

##### DNN 기반 회귀 모델 구현
- dataset : iris.csv
- feature : 3개, Sepal_Length, Sepal_Width, Petal_Length, Petal_width
- target : Setosa 외 나머지
- 학습방법 : 지도학습 > 분류 > 이진분류
- 알고리즘 : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
- 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비 <hr>

In [176]:
# 모듈 로딩
# - Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.classification import F1Score, BinaryF1Score
from torchinfo import summary

# - Data 및 시각화 관련
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [177]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수 구현!
print(f"Pytorch v.{torch.__version__}")
print(f"Pandas v.{pd.__version__}")

Pytorch v.2.4.1
Pandas v.2.0.3


In [178]:
### 데이터 로딩
DATA_FILE = '../../Data/iris.csv'

### CSV >>> DataFrame
irisDF = pd.read_csv(DATA_FILE)

### 확인
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa


In [179]:
torch.manual_seed(1)

In [180]:
### 타겟 변경 => 정수화, 클래스 3개 => 2개
irisDF['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [181]:
# -- 클래스 3개 => 2개
irisDF['variety'] = (irisDF['variety'] == 'Setosa')
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,True


In [182]:
irisDF['variety'] = irisDF['variety'].astype(int)

In [183]:
print(f"고유값 : {irisDF['variety'].unique()}")

고유값 : [1 0]


In [184]:
### 타겟 정수화
labels = dict(zip(irisDF['variety'].unique().tolist(),range(3)))
print(f"labels => {labels}")
# irisDF['variety] = irisDF['variety].replace(labels)
# irisDF.head(1)

labels => {1: 0, 0: 1}


[2] 모델 클래스 설계 및 정의 <hr> 
- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisBCFModel
- 부모 클래스 : nn.Module
- 배개   변수 : 층별 입출력 개수 고정하기 때문에 필요 없음!
- 속성 / 필드 : 
- 기능 / 역할 : __init__() : 모델 구조 설정, forward() : 순방향 학습 <= 오버라이딩(overriding)
- 클래스 구조 
    * 입력층 : 입력 4개(피쳐)     출력  10개 (퍼셉트론 / 뉴런 10개 존재)
    * 은닉층 : 입력 10개          출력  5개 (퍼셉트론 / 뉴런 5개 존재)
    * 출력층 : 입력 5개          출력  1개  (퍼셉트론 / 뉴런 1개 존재 : 2진 분류)

- 손실함수 / 활성화함수
    * 클래스 형태 ==> nn.MESLoss, nn.ReLU ==> init() 메서드
    * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드 

In [185]:
class IrisBCFModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(4, 10)
        self.hd_layer = nn.Linear(10, 5)
        self.out_layer = nn.Linear(5, 1)

    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # - 입력층
        y = self.in_layer(input_data)   # f11W11 + f12W12 + ... + f110W110 + b
        y = F.relu(y)                   # relu => y 값의 범위 : 0 <= y

        # - 은닉층 : 10개의 숫자 값(>= 0)
        y = self.hd_layer(y)            # f21W21 + f22W22 + ... + f230W230 + b
        y = F.relu(y)                   # relu => y 값의 범위 : 0 <= y

        # - 출력층 : 30개의 숫자 값(>= 0) 회귀이므로 바로 반환(return)
        return F.sigmoid(self.out_layer(y))          
        

In [186]:
### [테스트] 모델 인스턴스 생성
model = IrisBCFModel()
print(model)

IrisBCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=1, bias=True)
)


In [187]:
### [테스트] 모델 사용 메모리 정보 확인
summary(model, input_size = (100000, 4))

Layer (type:depth-idx)                   Output Shape              Param #
IrisBCFModel                             [100000, 1]               --
├─Linear: 1-1                            [100000, 10]              50
├─Linear: 1-2                            [100000, 5]               55
├─Linear: 1-3                            [100000, 1]               6
Total params: 111
Trainable params: 111
Non-trainable params: 0
Total mult-adds (M): 11.10
Input size (MB): 1.60
Forward/backward pass size (MB): 12.80
Params size (MB): 0.00
Estimated Total Size (MB): 14.40

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐 개수 : 3개
- 타겟 개수 : 1개
- 클래스 이름 : IrisDataset
- 부모 클래스 : utils.data.Dataset
- 속성 / 필드 : featureDF, targetDF, n_rows, n_features
- 필수 메서드 : 
    * _ _ init _ _(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * _ _ len_ _(self) : 데이터의 개수 반환
    * _ _ getitem _ _(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [188]:
class IrisDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]
    
    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        
        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [189]:
## 데이터셋 인스턴스 생성

# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]  # 2D (150, 3)
targetDF = irisDF[irisDF.columns[-1:]]   # 2D (150, 1)

# - 커스텀 데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF, targetDF)

# - 데이터로더 인스턴스 생성 
irisDL = DataLoader(irisDS)

for feature, target in irisDL:
    print(feature.shape, target.shape, feature, target)
    break

torch.Size([1, 4]) torch.Size([1, 1]) tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[1.]])


[4] 학습 준비
- 학습 횟수 : EPOCH         <- 처음 ~ 끝까지 공부하는 단위
- 배치 크기 : BATCH_SIZE    <- 한번에 학습할 데이터셋 양
- 위치 지정 : DEVICE        <- 텐서 저장 및 실행 위치 (GPU/CPU)
- 학습률    : 가중치와 절편 업데이트 시 경사 하강법으로 업데이트 간격 설정 0.001 ~ 0.1


In [190]:
### 학습 진행 관련 설정
EPOCH = 1000
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0] / BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

- 인스턴스 / 객체 : 모델, 데이터셋, 최적화(, 손실함수, 성능지표)

In [191]:
# 모델 인스턴스
model = IrisBCFModel()

In [192]:
# DS과 DL 인스턴스
# - 학습용, 검증용, 테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state = 1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = 1)
print(F'{X_train.shape} {X_test.shape} {X_val.shape}')
print(F'{y_train.shape} {y_test.shape} {y_val.shape}')
print(F'{type(X_train)} {type(X_test)} {type(X_val)}')

# - 학습용, 검증용, 테스트용 데이터셋
trainDS = IrisDataset(X_train, y_train)
valDS = IrisDataset(X_val, y_val)
testDF = IrisDataset(X_test, y_test)

# - 학습용 데이터로더 인스턴스
trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE, drop_last = True)

(84, 4) (38, 4) (28, 4)
(84, 1) (38, 1) (28, 1)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


- 최적화, 손실함수 인스턴스 생성

In [193]:
# 최적화 인스턴스 => W, b 텐서 즉, model.parameters() 전달
optimizer = optim.Adam(model.parameters(), lr = LR)

# 손실함수 인스턴스 => 분류 => 이진분류 BinaryCrossEntropyLoss => BCELoss
#                            예측값은 확률값으로 전달 ==> Sigmoid() AF 처리 후 전달
regLoss = nn.BCELoss()

In [194]:
len(trainDL), trainDL.__len__()

(9, 9)

[5] 학습 진행

In [195]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[], []], [[], []]
CNT = len(trainDL)

for epoch in range(EPOCH):
    # 배치 크기만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0, 0

    # 학습 모드로 모델 설정
    model.train()

    for featureTS, targetTS in trainDL:

        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산
        loss = regLoss(pre_y, targetTS)
        loss_total += loss.item()

        # 성능평가 계산
        score = BinaryF1Score()(pre_y, targetTS)
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증 기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)
        
        # 추론 / 평가
        pre_val = model(val_featureTS)
 
        # 손실
        loss_val = regLoss(pre_val, val_targetTS)
        
        # 성능 평가
        score_val = BinaryF1Score()(pre_val, val_targetTS)
        

    # 에포크 당 손실값과 성능평가 값 저장
    LOSS_HISTORY[0].append(loss_total / CNT)
    SCORE_HISTORY[0].append(score_total / CNT)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    print(f'[{epoch} / {EPOCH}]\n - [TRAIN] LOSS : {LOSS_HISTORY[0][-1]} SCORE : {SCORE_HISTORY[0][-1]}')
    print(F" - [VALID] LOSS : {LOSS_HISTORY[1][-1]} SCORE : {SCORE_HISTORY[1][-1]}")

[0 / 1000]
 - [TRAIN] LOSS : 0.6291880342695448 SCORE : 0.0
 - [VALID] LOSS : 0.7183738350868225 SCORE : 0.0
[1 / 1000]
 - [TRAIN] LOSS : 0.6076524588796828 SCORE : 0.0
 - [VALID] LOSS : 0.707494854927063 SCORE : 0.0
[2 / 1000]
 - [TRAIN] LOSS : 0.591214292579227 SCORE : 0.0
 - [VALID] LOSS : 0.6989750862121582 SCORE : 0.0
[3 / 1000]
 - [TRAIN] LOSS : 0.5778794454203712 SCORE : 0.0
 - [VALID] LOSS : 0.6914654970169067 SCORE : 0.0
[4 / 1000]
 - [TRAIN] LOSS : 0.5665415989028083 SCORE : 0.0
 - [VALID] LOSS : 0.6843708157539368 SCORE : 0.0
[5 / 1000]
 - [TRAIN] LOSS : 0.5566981103685167 SCORE : 0.0
 - [VALID] LOSS : 0.6770319938659668 SCORE : 0.0
[6 / 1000]
 - [TRAIN] LOSS : 0.5476581520504422 SCORE : 0.0
 - [VALID] LOSS : 0.6689302325248718 SCORE : 0.0
[7 / 1000]
 - [TRAIN] LOSS : 0.5389696823226081 SCORE : 0.0
 - [VALID] LOSS : 0.6599154472351074 SCORE : 0.0
[8 / 1000]
 - [TRAIN] LOSS : 0.5303041868739657 SCORE : 0.0
 - [VALID] LOSS : 0.6499184370040894 SCORE : 0.0
[9 / 1000]
 - [TRAIN]

- 학습결과 체크 => 학습과 